# Video Denoising Pipeline
### Classical DSP vs. Deep Learning (DnCNN) — Frame-by-Frame Video Processing

This notebook builds a complete video denoising pipeline targeting the following skills:
- **Data pipeline** for training, validation, and testing
- **Classical DSP baselines**: Gaussian, Wiener, Total Variation
- **Deep learning denoiser**: DnCNN architecture in PyTorch
- **Quantitative evaluation**: PSNR & SSIM comparison
- **Video processing**: frame-by-frame inference and video reconstruction

---
**Dependencies:**
```
pip install torch torchvision numpy scipy matplotlib scikit-image opencv-python Pillow
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from scipy import ndimage, signal
from skimage import data as skdata, color, util, restoration, filters
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage.transform import resize
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')
print(f'PyTorch: {torch.__version__}')

---
## 1 · Data Pipeline

We use built-in `skimage` images as clean reference frames. In production you would
swap in a real video dataset (e.g. DAVIS, Vimeo-90K).

Pipeline:
1. Load clean frames → normalize to [0, 1]
2. Add synthetic Gaussian noise → noisy frames
3. Wrap in a `torch.utils.data.Dataset`
4. Split train / val / test

In [ ]:
# ── Parameters ────────────────────────────────────────────────────────────────
NOISE_STD   = 25 / 255.0   # ← tune: noise level (25/255 is visually noticeable)
PATCH_SIZE  = 64            # ← tune: training patch size
BATCH_SIZE  = 32
NUM_EPOCHS  = 20            # ← tune: increase for better results
LR          = 1e-3

# ── Load clean images ─────────────────────────────────────────────────────────
def load_clean_frames():
    """Load skimage sample images as grayscale float32 frames."""
    loaders = [
        skdata.astronaut, skdata.camera, skdata.coins, skdata.horse,
        skdata.hubble_deep_field, skdata.moon, skdata.page, skdata.text,
        skdata.chelsea, skdata.coffee, skdata.immunohistochemistry,
        skdata.rocket
    ]
    frames = []
    for loader in loaders:
        img = loader()
        if img.ndim == 3:
            img = color.rgb2gray(img)
        img = resize(img, (256, 256), anti_aliasing=True)
        frames.append(img.astype(np.float32))
    return frames

clean_frames = load_clean_frames()
print(f'Loaded {len(clean_frames)} clean frames — shape: {clean_frames[0].shape}')

# Preview
fig, axes = plt.subplots(2, 6, figsize=(18, 6))
for ax, frame in zip(axes.flat, clean_frames):
    ax.imshow(frame, cmap='gray')
    ax.axis('off')
fig.suptitle('Clean Reference Frames', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
class VideoDenoiseDataset(Dataset):
    """Extracts random patches from clean frames and adds Gaussian noise."""

    def __init__(self, frames, patch_size=64, patches_per_frame=50, noise_std=25/255.0):
        self.patches     = []
        self.patch_size  = patch_size
        self.noise_std   = noise_std

        for frame in frames:
            H, W = frame.shape
            for _ in range(patches_per_frame):
                r = np.random.randint(0, H - patch_size)
                c = np.random.randint(0, W - patch_size)
                patch = frame[r:r+patch_size, c:c+patch_size]
                self.patches.append(patch)

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        clean = self.patches[idx]
        noise = np.random.normal(0, self.noise_std, clean.shape).astype(np.float32)
        noisy = np.clip(clean + noise, 0, 1)
        # Add channel dim: (1, H, W)
        clean_t = torch.from_numpy(clean).unsqueeze(0)
        noisy_t = torch.from_numpy(noisy).unsqueeze(0)
        noise_t = torch.from_numpy(noise).unsqueeze(0)  # DnCNN predicts noise
        return noisy_t, clean_t, noise_t

# ── Train / val / test split ──────────────────────────────────────────────────
train_frames = clean_frames[:8]
val_frames   = clean_frames[8:10]
test_frames  = clean_frames[10:]

train_ds = VideoDenoiseDataset(train_frames, PATCH_SIZE, patches_per_frame=100, noise_std=NOISE_STD)
val_ds   = VideoDenoiseDataset(val_frames,   PATCH_SIZE, patches_per_frame=50,  noise_std=NOISE_STD)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f'Train patches: {len(train_ds)} | Val patches: {len(val_ds)}')

---
## 2 · Classical DSP Baselines

Before training the CNN, we establish classical denoising baselines to beat.

In [ ]:
def add_noise(img, std=NOISE_STD):
    noise = np.random.normal(0, std, img.shape).astype(np.float32)
    return np.clip(img + noise, 0, 1)

def classical_denoise(noisy):
    return {
        'Gaussian':       filters.gaussian(noisy, sigma=1.5),
        'Wiener':         signal.wiener(noisy, mysize=5),
        'Total Variation': restoration.denoise_tv_chambolle(noisy, weight=0.1),
    }

def compute_metrics(clean, candidate):
    p = psnr(clean, candidate, data_range=1.0)
    s = ssim(clean, candidate, data_range=1.0)
    return p, s

# Evaluate on test frames
test_img   = test_frames[0]
noisy_img  = add_noise(test_img)
results    = classical_denoise(noisy_img)

print(f'  Noisy input       PSNR={compute_metrics(test_img, noisy_img)[0]:.2f} dB  SSIM={compute_metrics(test_img, noisy_img)[1]:.4f}')
for name, denoised in results.items():
    p, s = compute_metrics(test_img, denoised)
    print(f'  {name:<20} PSNR={p:.2f} dB  SSIM={s:.4f}')

# Visualize
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for ax, (img, title) in zip(axes, [
    (test_img,  'Clean'),
    (noisy_img, f'Noisy (σ={NOISE_STD:.3f})'),
    *[(v, k) for k, v in results.items()]
]):
    ax.imshow(img, cmap='gray')
    ax.set_title(title)
    ax.axis('off')
fig.suptitle('Classical DSP Baselines', fontweight='bold')
plt.tight_layout()
plt.show()

---
## 3 · DnCNN — Deep Learning Denoiser

**DnCNN** (Zhang et al., 2017) uses residual learning: the network predicts
the **noise** rather than the clean image.

```
clean = noisy − DnCNN(noisy)
```

Architecture:
- **Layer 1**: Conv(64) + ReLU
- **Layers 2–(D-1)**: Conv(64) + BatchNorm + ReLU
- **Layer D**: Conv(1) — outputs predicted noise map

In [ ]:
class DnCNN(nn.Module):
    """DnCNN: Beyond Gaussian Denoiser (Zhang et al., 2017)."""

    def __init__(self, depth=17, n_filters=64, kernel_size=3):
        super().__init__()
        pad = kernel_size // 2
        layers = []

        # First layer: Conv + ReLU (no BN)
        layers += [nn.Conv2d(1, n_filters, kernel_size, padding=pad, bias=False),
                   nn.ReLU(inplace=True)]

        # Middle layers: Conv + BN + ReLU
        for _ in range(depth - 2):
            layers += [nn.Conv2d(n_filters, n_filters, kernel_size, padding=pad, bias=False),
                       nn.BatchNorm2d(n_filters),
                       nn.ReLU(inplace=True)]

        # Last layer: Conv only — outputs noise map
        layers += [nn.Conv2d(n_filters, 1, kernel_size, padding=pad, bias=False)]

        self.net = nn.Sequential(*layers)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        noise_pred = self.net(x)
        return x - noise_pred          # residual: predicted clean image

model = DnCNN(depth=17).to(DEVICE)
total_params = sum(p.numel() for p in model.parameters())
print(f'DnCNN parameters: {total_params:,}')
print(model)

---
## 4 · Training Pipeline

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

train_losses, val_losses = [], []

for epoch in range(NUM_EPOCHS):
    # ── Train ──────────────────────────────────────────────────────────────────
    model.train()
    t_loss = 0.0
    for noisy, clean, _ in train_loader:
        noisy, clean = noisy.to(DEVICE), clean.to(DEVICE)
        optimizer.zero_grad()
        output = model(noisy)
        loss   = criterion(output, clean)
        loss.backward()
        optimizer.step()
        t_loss += loss.item()

    # ── Validate ───────────────────────────────────────────────────────────────
    model.eval()
    v_loss = 0.0
    with torch.no_grad():
        for noisy, clean, _ in val_loader:
            noisy, clean = noisy.to(DEVICE), clean.to(DEVICE)
            output = model(noisy)
            v_loss += criterion(output, clean).item()

    t_loss /= len(train_loader)
    v_loss /= len(val_loader)
    train_losses.append(t_loss)
    val_losses.append(v_loss)
    scheduler.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1:>3}/{NUM_EPOCHS}]  Train Loss: {t_loss:.6f}  Val Loss: {v_loss:.6f}  LR: {scheduler.get_last_lr()[0]:.2e}')

# ── Plot loss curves ──────────────────────────────────────────────────────────
plt.figure(figsize=(8, 4))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses,   label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('DnCNN Training Curves')
plt.legend()
plt.tight_layout()
plt.show()

---
## 5 · Evaluation — Classical vs. DnCNN

In [ ]:
def dncnn_denoise(model, noisy_img):
    """Run DnCNN inference on a single grayscale image."""
    model.eval()
    with torch.no_grad():
        t = torch.from_numpy(noisy_img).unsqueeze(0).unsqueeze(0).to(DEVICE)
        out = model(t)
        out = out.squeeze().cpu().numpy()
    return np.clip(out, 0, 1)

print('\n=== Denoising Benchmark (test frames) ===')
print(f'{"Method":<22} {"PSNR (dB)":>10} {"SSIM":>8}')
print('-' * 42)

all_results = {}
for test_img in test_frames:
    noisy = add_noise(test_img)
    classical = classical_denoise(noisy)
    cnn_out   = dncnn_denoise(model, noisy.astype(np.float32))

    methods = {'Noisy': noisy, **classical, 'DnCNN': cnn_out}
    for name, img in methods.items():
        p, s = compute_metrics(test_img, img)
        all_results.setdefault(name, []).append((p, s))

for name, scores in all_results.items():
    avg_p = np.mean([s[0] for s in scores])
    avg_s = np.mean([s[1] for s in scores])
    print(f'{name:<22} {avg_p:>10.2f} {avg_s:>8.4f}')

In [ ]:
# ── Visual comparison on one test frame ───────────────────────────────────────
test_img  = test_frames[0]
noisy_img = add_noise(test_img)
classical = classical_denoise(noisy_img)
cnn_out   = dncnn_denoise(model, noisy_img.astype(np.float32))

imgs   = [test_img, noisy_img, *classical.values(), cnn_out]
titles = ['Clean', f'Noisy (σ={NOISE_STD:.3f})', *classical.keys(), 'DnCNN']

fig, axes = plt.subplots(1, len(imgs), figsize=(4 * len(imgs), 4))
for ax, img, title in zip(axes, imgs, titles):
    p, s = compute_metrics(test_img, img)
    ax.imshow(img, cmap='gray')
    ax.set_title(f'{title}\nPSNR={p:.1f} dB', fontsize=9)
    ax.axis('off')
fig.suptitle('Classical DSP vs. DnCNN', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 6 · Video Processing Pipeline

Apply the trained DnCNN frame-by-frame to a video.
- **Option A**: Use the synthetic video generated below
- **Option B**: Swap in your own `.mp4` path

In [ ]:
# ── Option A: Generate a synthetic noisy video from test frames ───────────────
SYNTHETIC_VIDEO = 'synthetic_noisy.avi'
H, W = 256, 256
fps  = 10

writer = cv2.VideoWriter(SYNTHETIC_VIDEO, cv2.VideoWriter_fourcc(*'XVID'), fps, (W, H), isColor=False)
source_frames = []
for frame in test_frames * 5:   # repeat to make a longer clip
    noisy = add_noise(frame)
    uint8 = (noisy * 255).astype(np.uint8)
    writer.write(uint8)
    source_frames.append((frame, noisy))
writer.release()
print(f'Synthetic video saved: {SYNTHETIC_VIDEO}  ({len(source_frames)} frames)')

# ── Option B: use your own video ──────────────────────────────────────────────
# SYNTHETIC_VIDEO = 'your_video.mp4'

In [ ]:
def process_video(input_path, output_path, model, noise_std=None):
    """
    Read a video frame-by-frame, denoise with DnCNN, write output video.
    If noise_std is set, synthetic noise is added before denoising.
    """
    cap  = cv2.VideoCapture(input_path)
    fps  = cap.get(cv2.CAP_PROP_FPS)
    W    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H    = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (W, H), isColor=False)

    frame_psnrs = []
    model.eval()

    for i in range(total):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale float
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0

        if noise_std:
            gray = np.clip(gray + np.random.normal(0, noise_std, gray.shape).astype(np.float32), 0, 1)

        # DnCNN inference
        denoised = dncnn_denoise(model, gray)
        denoised_u8 = (denoised * 255).astype(np.uint8)
        out.write(denoised_u8)

    cap.release()
    out.release()
    print(f'Denoised video saved: {output_path}  ({total} frames)')

process_video(SYNTHETIC_VIDEO, 'denoised_output.avi', model)

# ── Visualize sample frames ───────────────────────────────────────────────────
cap = cv2.VideoCapture(SYNTHETIC_VIDEO)
sample_noisy, sample_denoised = [], []
for i in range(5):
    ret, frame = cap.read()
    if not ret: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
    sample_noisy.append(gray)
    sample_denoised.append(dncnn_denoise(model, gray))
cap.release()

fig, axes = plt.subplots(2, 5, figsize=(18, 6))
for i in range(5):
    axes[0, i].imshow(sample_noisy[i],    cmap='gray'); axes[0, i].set_title(f'Noisy frame {i+1}');    axes[0, i].axis('off')
    axes[1, i].imshow(sample_denoised[i], cmap='gray'); axes[1, i].set_title(f'Denoised frame {i+1}'); axes[1, i].axis('off')
fig.suptitle('Video Frame Denoising — DnCNN', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 7 · Save & Load Model

In [ ]:
# ── Save ──────────────────────────────────────────────────────────────────────
torch.save(model.state_dict(), 'dncnn_weights.pth')
print('Model saved to dncnn_weights.pth')

# ── Load (example) ────────────────────────────────────────────────────────────
# model_loaded = DnCNN(depth=17).to(DEVICE)
# model_loaded.load_state_dict(torch.load('dncnn_weights.pth', map_location=DEVICE))
# model_loaded.eval()